# Test Analystic GLRT vs Optimization GLRT

In [1]:
# Comparing the analytic GLRT for the single-coefficient case with the optimization-based version
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
from uncertainty.DataGeneration import linearRegression_normal
from uncertainty.torch_linear import TorchLinear
from uncertainty.analyticLinearRegressionCIs import analyticLinearTest_GLRT, analyticLinearCis
from uncertainty.glrtTorch import glrtTorchCis
from uncertainty.torch_linear import TorchLinear

In [3]:
def compareAnalyticVsOptimization(beta, n=200, alpha=0.05):
    """Generate data according to a linear regression with coefficient vector beta.
    Compute the analytic chi-squared test 'is the coefficient vector zero?'. Record the p-value.
    Compute the confidence intervals on beta using the GLRT-based method. Record whether ALL intervals contains zero.
    Return TRUE if both methods agree, and FALSE otherwise."""  
    # Get data
    X, y = linearRegression_normal(beta=np.array(beta), cov=np.eye(len(beta)), sigma=1, n=n)

    # Compute the p-value from the chi squared test
    TL = TorchLinear(lr=0.3,max_iter=30)
    TL.fit(X,y)
    pVal_TL = analyticLinearTest_GLRT(TL, X, y)
    
    # Next, get the optimization-based intervals.
    # GLRT method with Torch model (coefs)
    lcb_GLRT, ucb_GLRT, lcb_Results, ucb_Results, ucb_Torch = glrtTorchCis(
        lambda:TorchLinear(lr=0.3,max_iter=100,fit_intercept=False), X=X, y=y, citype='coefs', alpha=alpha,
        search_kwargs={'lmbds':np.logspace(-10,10,101)},fit_kwargs={'lr':0.3,'max_iter':30})
    
    # To check whether they agree with the analytic version:
    # The p-value should be < 0.05 iff any of the level-0.05 glrt-based confidence intervals
    # on the coefficient beta don't contain zero.
    isNotZero_analytic = pVal_TL < alpha    # The chi-square test is one-sided
    isNotZero_optim = any(lcb_GLRT > 0) or any(ucb_GLRT < 0)
    
    print("p value", pVal_TL, "and confidence interval(s)", lcb_GLRT, "to", ucb_GLRT)
    
    return (isNotZero_analytic == isNotZero_optim, pVal_TL, lcb_GLRT, ucb_GLRT)
    

## Single nonzero coefficient

In [ ]:
%%time
beta = [0.1]

success = []
pVals = []
lcbs = []
ucbs = []

#for i in range(1000):
for i in range(100):
    s, p, l, u = compareAnalyticVsOptimization(beta=beta)
    success.append(s)
    pVals.append(p)
    lcbs.append(l)
    ucbs.append(u)

p value 0.2730769739905715 and confidence interval(s) [-0.04621225] to [0.18728517]
p value 0.18350520501753576 and confidence interval(s) [-0.03571206] to [0.22157654]
p value 0.03294902462555194 and confidence interval(s) [0.03522803] to [0.25493419]
p value 0.02266104231696031 and confidence interval(s) [0.03810236] to [0.28994715]
p value 0.112772854276467 and confidence interval(s) [-0.01754824] to [0.24477921]
p value 0.20790656977214939 and confidence interval(s) [0.0134417] to [0.16700079]
p value 0.7055638248940208 and confidence interval(s) [-0.1546178] to [0.09814219]
p value 0.9937585163692888 and confidence interval(s) [-0.11516205] to [0.11627033]
p value 0.08109185593955626 and confidence interval(s) [0.01012423] to [0.21824399]
p value 0.11871346287756657 and confidence interval(s) [-0.01366923] to [0.23345451]
p value 0.5109407684841294 and confidence interval(s) [-0.07371692] to [0.16548997]
p value 0.34044925025777195 and confidence interval(s) [-0.05463722] to [0.18

In [ ]:
minY = min(lcbs)
maxY = max(ucbs)
plt.ylim(minY, maxY)

for (p, l, u, s) in zip(pVals, lcbs, ucbs, success):
    # ymin and ymax are the 'fraction of the way up from the bottom of the plot'
    plt.axvline(x=p, ymin=(l-minY)/(maxY - minY), ymax=(u-minY)/(maxY-minY),
                color=('darkgreen' if s else 'darkred'), alpha=0.2)

# plt.scatter(np.asarray(pVals)[np.array(success)], np.asarray(lcbs)[np.array(success)], color='darkgreen')
# plt.scatter(np.asarray(pVals)[np.array(success)], np.asarray(ucbs)[np.array(success)], color='darkgreen')

# plt.scatter(np.asarray(pVals)[~np.array(success)], np.asarray(lcbs)[~np.array(success)], color='darkred', marker='x')
# plt.scatter(np.asarray(pVals)[~np.array(success)], np.asarray(ucbs)[~np.array(success)], color='darkred', marker='x')

plt.axhline(0, color='k', linestyle='--')
plt.axvline(0.05, color='gray', linestyle='--')

plt.xlabel("p-value")
plt.ylabel("Range of confidence interval")
plt.title("Succeeded in " + str(np.mean(success)*100) + "% of trials")

plt.show()

## Single Zero Coefficient

In [ ]:
%%time
beta = [0.0]

success = []
pVals = []
lcbs = []
ucbs = []

for i in range(1000):
#for i in range(100):
    s, p, l, u = compareAnalyticVsOptimization(beta=beta)
    success.append(s)
    pVals.append(p)
    lcbs.append(l)
    ucbs.append(u)

In [ ]:
minY = min(lcbs)
maxY = max(ucbs)
plt.ylim(minY, maxY)

for (p, l, u, s) in zip(pVals, lcbs, ucbs, success):
    # ymin and ymax are the 'fraction of the way up from the bottom of the plot'
    plt.axvline(x=p, ymin=(l-minY)/(maxY - minY), ymax=(u-minY)/(maxY-minY),
                color=('darkgreen' if s else 'darkred'), alpha=0.2)

# plt.scatter(np.asarray(pVals)[np.array(success)], np.asarray(lcbs)[np.array(success)], color='darkgreen')
# plt.scatter(np.asarray(pVals)[np.array(success)], np.asarray(ucbs)[np.array(success)], color='darkgreen')

# plt.scatter(np.asarray(pVals)[~np.array(success)], np.asarray(lcbs)[~np.array(success)], color='darkred', marker='x')
# plt.scatter(np.asarray(pVals)[~np.array(success)], np.asarray(ucbs)[~np.array(success)], color='darkred', marker='x')

plt.axhline(0, color='k', linestyle='--')
plt.axvline(0.05, color='gray', linestyle='--')

plt.xlabel("p-value")
plt.ylabel("Range of confidence interval")
plt.title("Succeeded in " + str(np.mean(success)*100) + "% of trials")

plt.show()

## Multi-dimensional Beta

In [ ]:
%%time
beta = [0.0, 0.0]

success = []
pVals = []
lcbs = []
ucbs = []

for i in range(1000):
#for i in range(100):
    s, p, l, u = compareAnalyticVsOptimization(beta=beta)
    success.append(s)
    pVals.append(p)
    lcbs.append(l)
    ucbs.append(u)

In [ ]:
d = len(beta)
fig, ax = plt.subplots(d)
for i in range(d):
    ilcbs = [l[i] for l in lcbs]
    iucbs = [u[i] for u in ucbs]
    minY = min(ilcbs)
    maxY = max(iucbs)
    ax[i].set_ylim(minY, maxY)
    
    for (p, l, u, s) in zip(pVals, ilcbs, iucbs, success):
        # ymin and ymax are the 'fraction of the way up from the bottom of the plot'
        ax[i].axvline(x=p, ymin=(l-minY)/(maxY - minY), ymax=(u-minY)/(maxY-minY),
                    color=('darkgreen' if s else 'darkred'), alpha=0.3)

    ax[i].axhline(0, color='k', linestyle='--')
    ax[i].axvline(0.05, color='gray', linestyle='--')
    
ax[0].set_title("Succeeded in " + str(np.mean(success)*100) + "% of trials")    
ax[d-1].set_xlabel("p-value")
ax[0].set_ylabel("Range of confidence interval")
plt.tight_layout()
plt.show()

## Higher dimensional beta - check for multiple testing problems (I don't think we will have them... but good to confirm)

In [ ]:
%%time
beta = [0.0, 0.0, 0.0, 0.0, 0.0]

success = []
pVals = []
lcbs = []
ucbs = []

for i in range(1000):
#for i in range(100):
    s, p, l, u = compareAnalyticVsOptimization(beta=beta)
    success.append(s)
    pVals.append(p)
    lcbs.append(l)
    ucbs.append(u)

In [ ]:
d = len(beta)
fig, ax = plt.subplots(d)
for i in range(d):
    ilcbs = [l[i] for l in lcbs]
    iucbs = [u[i] for u in ucbs]
    minY = min(ilcbs)
    maxY = max(iucbs)
    ax[i].set_ylim(minY, maxY)
    
    for (p, l, u, s) in zip(pVals, ilcbs, iucbs, success):
        # ymin and ymax are the 'fraction of the way up from the bottom of the plot'
        ax[i].axvline(x=p, ymin=(l-minY)/(maxY - minY), ymax=(u-minY)/(maxY-minY),
                    color=('darkgreen' if s else 'darkred'), alpha=0.3)

    ax[i].axhline(0, color='k', linestyle='--')
    ax[i].axvline(0.05, color='gray', linestyle='--')
    
ax[0].set_title("Succeeded in " + str(np.mean(success)*100) + "% of trials")    
ax[d-1].set_xlabel("p-value")
ax[0].set_ylabel("Range of confidence interval")
plt.tight_layout()
plt.show()